In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [4]:
from typing import Optional, List, Dict, Tuple

In [56]:
from configs.config import cfg, get_cfg_defaults
from core.datasets.dataset_loading_utils import load_dataset
from core.datasets.vq_dataset import DATALoader
from utils.vis_utils import plot_3d_global
from core.models.conv_vqvae import ConvVQMotionModel

In [90]:
cfg = get_cfg_defaults()
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/convq_256_1024_affine/convq_256_1024_affine.yaml")

In [56]:
vqvae_model = ConvVQMotionModel(
            cfg.vqvae,
            # is_distributed=self.is_distributed,
        )

Sync is turned on False


In [58]:
batch["motion"].shape

torch.Size([1, 80, 271])

In [57]:
a,b,c = vqvae_model(batch["motion"])

In [65]:
len(b.flatten().tolist())

20

In [68]:
np.unique(b.flatten().tolist())

array([ 88, 226, 248, 267, 332, 369, 425, 557, 581, 591, 609, 692, 704,
       737, 754, 815, 845, 930, 935, 951])

## Dataloader

In [62]:
render_ds, sampler_val, weights_val = load_dataset(
                ["t2m", "aist", "cm"],cfg, "render"
            )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 44.83it/s]


Total number of motions 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 56.44it/s]


Total number of motions 2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 273.46it/s]

Total number of motions 3


In [63]:
render_dl = DATALoader(
    
            render_ds, batch_size=1, shuffle=False
        )

In [59]:
batch = (next(iter(render_dl)))
    
print(batch["names"])

['00_1167']


In [50]:
from utils.vis_utils.render_final import Renderer
renderer = Renderer("cuda")

In [14]:
from pathlib import Path


In [43]:
def load(vqvae_model,  path):
    path = Path(path)
    assert path.exists()
    pkg = torch.load(str(path), map_location="cuda")
    vqvae_model.vq._codebook.batch_mean = pkg["model"][
        "vq._codebook.batch_mean"
    ]
    vqvae_model.vq._codebook.batch_variance = pkg["model"][
        "vq._codebook.batch_variance"
    ]

    vqvae_model.load_state_dict(pkg["model"])
    vqvae_model = vqvae_model.to(device)

In [6]:
from core.models.conformer_vqvae import ConformerVQMotionModel

In [7]:
cfg = get_cfg_defaults()
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml")

In [9]:
model = ConformerVQMotionModel(cfg.vqvae)

Sync is turned on False


In [16]:
save_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/checkpoints/vqvae_motion.320000.pt"

In [44]:
load(model,save_path)
model = model.eval()

In [79]:
aa = model.motionEncoder( torch.randn((1, 80 , 271)).cuda() , True)

In [92]:
embed_motion_features = aa.mean() + torch.randn((1, 20 , 768)).to(model.device) * aa.var()

In [93]:
quantized_enc_motion, indices, commit_loss = model.vq(embed_motion_features)

In [94]:
decoded_motion_features = model.motionDecoder(quantized_enc_motion, True)

In [95]:
render_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/renders/"

In [97]:
renderer.render(
                    motion_vec=render_ds.datasets[1].inv_transform(decoded_motion_features.detach().cpu().squeeze().float()),
                    outdir=render_path,
                    step=0,
                    name="rnd2",
                )